# Classificador KNN - "Vote"

Estamos interessados em predizer classes do data set "vote" (weka) utilizando nosso classificador KNN.

## Importando bibliotecas

In [ ]:
# Para obter o data set
from util.dataparser import DataParser as dp
# Biblioteca numpy
import numpy as np
# Classificador KNN
from classifiers.neighbors import KNeighborsClassifier
# Para medir o tempo de execução dos algoritmos
from ext.timer import elapsed_timer

## Obtendo o data set

In [ ]:
def get_dataset():
    X, y = dp.arff_data('datasets/vote.arff', attr_type=None)

    X = np.asarray(X)
    y = np.asarray(y)
    return X, y

X, y = get_dataset()

print("Instâncias:{}".format(len(X)))

Perceba que algumas instâncias do data set contém alguns valores de atributos faltando. Além disso, nossos atributos não são numericos, logo, precisamos realizar algum tipo de pré-processamento.

### Pré-processamento

Para lidar com os dados ausentes, podemos eliminar as instâncias com essa falta de dados, ou ainda poderíamos tratar esse problema com outra abordagem. Por ora, iremos eliminar as instâncias e analizar seus efeitos.

In [ ]:
missing = []
for i in range(len(X)):
    if '?' in X[i]:
        missing.append(i)
        
X = np.delete(X, missing, axis=0)
y = np.delete(y, missing, axis=0)

len(X)

Perceba que nosso data set praticamente diminuiu pela metade. Isso pode não ser viável, já que temos muitos atributos e a maldição da dimensionalidade pode se agravar nesse caso. 

Podemos substituir os valores ausentes pela média dos valores de todas as instâncias para aquele atributo. Mas primeiro, devemos obter o dataset original novamente e transformar nossos atributos nominais em numéricos.

In [ ]:
X, y = get_dataset()

Note que os dados do data set são binários, logo, podemos fazer  uma substituição simples de 'y' para 1, e 'n' para 0.

In [ ]:
X[X == "'y'"] = '1'
X[X == "'n'"] = '0'
X[X == '?'] = -1  # Iremos tratar este caso a seguir

X = X.astype(float)  # Casting dos valores para int

X[X == -1.] = np.nan

Agora precisamos calcular a média dos valores de cada atributo e substituir os dados ausentes.

In [ ]:
means = np.nanmean(X, axis=0)

for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        if np.isnan(X[i, j]):
            X[i, j] = means[j]
            
X

Permutamos o nosso data set para obter instâncias randomicas de treino e teste:

In [ ]:
indices = np.random.permutation(len(X))
X_train = X[indices[:-20]]
y_train = y[indices[:-20]]
X_test  = X[indices[-20:]]
y_test  = y[indices[-20:]]

X_train, y_train
X_test, y_test

## Classificação

### Definindo uma função que exibe os resultados

In [ ]:
def mostrar_resultados(X_test, pred):
    i = 0
    total_correct = 0
    for test in X_test:
        if pred[i] == y_test[i]:
            total_correct += 1
        acc = (total_correct / (i+1)) * 100
        print('test['+str(i)+']', '\tpred:', pred[i], '\torig:', y_test[i], '\tacc:', str(round(acc, 2))+'%')
        i += 1

### Classificando imagens 

#### Algoritmo KD-Tree

A seguir instanciamos o nosso classificador knn com o algoritmo _kd-tree_. 

Observe como a acurácia do data set de teste muda conforme o valor do tamanho da folha.

In [ ]:
# Testando com o tamanho de folha igual a 30
with elapsed_timer() as elapsed:
    classifier = KNeighborsClassifier(leaf_size=30, n_neighbors=5, algorithm='kd_tree')
    classifier.fit(X_train, y_train)
    pred = classifier.predict(X_test)
    print("Tempo de execução: " + str(elapsed()))

mostrar_resultados(y_test, pred)

In [ ]:
# Testando com o tamanho de folha igual a 100
with elapsed_timer() as elapsed:
    classifier = KNeighborsClassifier(leaf_size=100, n_neighbors=5, algorithm='kd_tree')
    classifier.fit(X_train, y_train)
    pred = classifier.predict(X_test)
    print("Tempo de execução: " + str(elapsed()))
    
mostrar_resultados(y_test, pred)

In [ ]:
# Testando com o tamanho de folha igual a 1000
with elapsed_timer() as elapsed:
    classifier = KNeighborsClassifier(leaf_size=1000, n_neighbors=5, algorithm='kd_tree')
    classifier.fit(X_train, y_train)
    pred = classifier.predict(X_test)
    print("Tempo de execução: " + str(elapsed()))   

mostrar_resultados(y_test, pred)

#### Algoritmo de força bruta

In [ ]:
with elapsed_timer() as elapsed:
    classifier = KNeighborsClassifier(algorithm='brute')
    classifier.fit(X_train, y_train)
    pred = classifier.predict(X_test)
    print("Tempo de execução: " + str(elapsed()))

mostrar_resultados(y_test, pred)

### Comparando resultados com sci-kit learn

O _sci-kit learn_ implementa o algoritmo de classificação KNN de forma extremamente eficiente. Observe os resultados obtidos ao classificar os dígitos usando a biblioteca ```sklearn.neighbors```.

In [ ]:
# Classificador KNN do sci-kit learn
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# Pelo algoritmo kd-tree:
with elapsed_timer() as elapsed:
    classifier = KNeighborsClassifier(algorithm='kd_tree')
    classifier.fit(X_train, y_train)
    pred = classifier.predict(X_test)
    print("Tempo de execução: " + str(elapsed()))

mostrar_resultados(y_test, pred)

In [ ]:
# Pelo algoritmo de força bruta:
with elapsed_timer() as elapsed:
    classifier = KNeighborsClassifier(algorithm='brute')
    classifier.fit(X_train, y_train)
    pred = classifier.predict(X_test)
    print("Tempo de execução: " + str(elapsed()))

mostrar_resultados(y_test, pred)

## Referencias

Rahul Bhalley, 2017. __Digit recognition__. _https://towardsdatascience.com/mnist-with-k-nearest-neighbors-8f6e7003fab7_. 